In [1]:
# To handle relative importing
import os
import sys
module_paths = [
    os.path.abspath(os.path.join('..')),
]
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

In [2]:
%load_ext autoreload
%autoreload 2
# import necessary files
from tqdm import tqdm
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
# dataloader
from src.dataloaders.dataloader import ImgNetDataset
from torch.utils.data import DataLoader

In [24]:
# imgnet 1k
dataset = ImgNetDataset("/root/CS570-Final-Project/datasets/imgnet1k_original/val.json", type='val')
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [21]:
from PIL import Image

# resnet50 torch

import time
from torchvision import transforms

resnet50 = torchvision.models.resnet50(pretrained=True)
resnet50.eval()
resnet50.cuda()

json_path = '/root/CS570-Final-Project/datasets/imgnet1k_original/val.json'
with open(json_path, 'r') as f:
    json_data = json.load(f)
    json_data = json_data['data']
    # random.shuffle(json_data)

acc, cnt = 0, 0
ms = 1000
B = 256
img_L, labels_L = [], []
for idx, item in enumerate(tqdm(json_data)):
    start_time = time.time()
    img_path = item['img_path']
    img = Image.open(img_path).convert("RGB")
    img = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(256, antialias=True),
                transforms.CenterCrop(224),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])(img).unsqueeze(0).cuda()
    end_time = time.time()
    print(end_time - start_time)

    labels = torch.tensor(item['labels'])
    y = torch.zeros(1000)
    if len(labels) > 0:
        y[labels] = 1

    y = y.unsqueeze(0).cuda()

    img_L.append(img)
    labels_L.append(y)

    if len(img_L) == B or idx == len(json_data) - 1:
        img = torch.cat(img_L, dim=0)
        y = torch.cat(labels_L, dim=0)    
        img_L, labels_L = [], []    
        with torch.no_grad():
            output = resnet50(img)
            p_idx = output.argmax(dim=1)
            gt_idx = y.argmax(dim=1)
            acc += (p_idx == gt_idx).sum().item()
            cnt += len(p_idx)
        if cnt > ms:
            print(acc / cnt)
            ms += 1000
print(acc / cnt)

/root/anaconda3/envs/cs570_t23/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/anaconda3/envs/cs570_t23/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 11/50000 [00:00<08:03, 103.42it/s]

0.011360406875610352
0.009021997451782227
0.00826263427734375
0.0052835941314697266
0.017299890518188477
0.008011817932128906
0.00715184211730957
0.005626201629638672
0.005816936492919922
0.006601572036743164
0.018922090530395508
0.005486488342285156
0.005463123321533203
0.01348567008972168
0.013441324234008789
0.006281852722167969
0.008848428726196289
0.008022069931030273
0.003916501998901367
0.005635261535644531
0.007680177688598633
0.006453275680541992
0.008424758911132812
0.009181976318359375


  0%|          | 36/50000 [00:00<08:04, 103.08it/s]

0.01577448844909668
0.008208513259887695
0.008304834365844727
0.013924598693847656
0.011993408203125
0.007631063461303711
0.012216567993164062
0.007970333099365234
0.009371519088745117
0.005208253860473633
0.015096426010131836
0.010918617248535156
0.02179551124572754
0.008940696716308594
0.008668661117553711
0.0070345401763916016
0.004587411880493164
0.005095243453979492
0.009090185165405273
0.007329702377319336
0.006368398666381836


  0%|          | 61/50000 [00:00<07:24, 112.32it/s]

...

  2%|▏         | 950/50000 [00:10<08:56, 91.36it/s]

0.01087808609008789
0.010535955429077148
0.008312702178955078
0.011071443557739258
0.011018514633178711
0.01100778579711914
0.013554573059082031
0.01273345947265625
0.016093015670776367
0.0046024322509765625
0.010533571243286133
0.016057252883911133
0.01145625114440918
0.009504318237304688
0.009164571762084961
0.009970426559448242
0.011039018630981445
0.013171672821044922


  2%|▏         | 960/50000 [00:10<09:05, 89.89it/s]


0.011579275131225586
0.009771347045898438
0.015502214431762695
0.008496284484863281
0.009202241897583008
0.006306171417236328
0.010746479034423828
0.008007526397705078
0.010454177856445312


KeyboardInterrupt: 

In [25]:
# resnet50 torch
resnet50 = torchvision.models.resnet50(pretrained=True)
resnet50.eval()
resnet50.cuda()

acc, cnt = 0, 0
ms = 1000
start_time = time.time()
for img, labels in tqdm(dataloader):
    img = img.cuda()
    labels = labels.cuda()
    end_time = time.time()
    # print(end_time - start_time)
    with torch.no_grad():
        output = resnet50(img)
        idx = output.argmax(dim=1)
        gt_idx = labels.argmax(dim=1)
        acc += (idx == gt_idx).sum().item()
        cnt += len(idx)
    if cnt > ms:
        print(acc / cnt)
        ms += 1000
    start_time = time.time()
print(acc / cnt)

  1%|          | 433/50000 [00:08<15:16, 54.10it/s]


KeyboardInterrupt: 